In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import wobble

In [ ]:
import matplotlib as mpl
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['figure.figsize'] = (12.0, 4.0)

#### First, you'll need some data to load up.

You can download example HARPS data files (and results files) to play around with linked in the [documentation](https://wobble.readthedocs.io/en/latest/quickstart.html#input-data).

Here we'll assume that you have the data `51peg_e2ds.hdf5` saved in the `wobble/data` directory.

By default, loading the data will load all echelle orders and all epochs in the data file; you can change this with the optional `orders` and `epochs` kwargs, which each take lists (or 1-d numpy arrays) of indices for the desired orders/epochs to load.

In [ ]:
data = wobble.Data('../data/51peg_e2ds.hdf5')

The data we just loaded are assumed to be continuum normalized, with regions of bad data (negative flux values or very low SNR) "masked out" by setting their uncertainties to be infinite. In this example, the data are also in units of log(wavelength) vs. log(flux).

In [ ]:
r = 0 # index of echelle order to plot
n = 0 # index of epoch to plot
plt.plot(data.xs[r][n], data.ys[r][n], 'k.', ms=6)
mask = data.ivars[r][n] <= 1.e-8 # masked-out bad data
plt.plot(data.xs[r][n][mask], data.ys[r][n][mask], 'w.', ms=4)
plt.ylabel('ln(flux)')
plt.xlabel('ln(wave)');

#### Now let's create a results object in which to store the outputs of wobble:

In [ ]:
results = wobble.Results(data=data)

This object is not currently populated with useful information (because we haven't optimized anything yet!), but once it is we'll be able to save it with the `results.write('filename.hdf5')` function.

A saved results file can be loaded as:
```results = wobble.Results(filename='filename.hdf5')```

### Here's a minimal example of optimizing a model consisting of a star and tellurics for a single order:

In [ ]:
r = 67 # index into data.orders for the desired order
model = wobble.Model(data, results, r)
model.add_star('star')
model.add_telluric('tellurics')
wobble.optimize_order(model)

The results have been automatically saved and we can now view them by generating a plot:

In [ ]:
n = 40 # epoch to plot
results.plot_spectrum(r, n, data, 'demo1.png')
from IPython.display import Image
Image(filename='demo1.png') 

The residuals look good for the star but not great around the tellurics. Let's try running with variable tellurics.

In [ ]:
results2 = wobble.Results(data=data)
model = wobble.Model(data, results2, r)
model.add_star('star')
model.add_telluric('tellurics', variable_bases=2)
wobble.optimize_order(model)

In [ ]:
results2.plot_spectrum(r, n, data, 'demo2.png')
Image(filename='demo2.png') 

Looks better! Here are the RVs for this single order. Once we run on all orders, we can combine the order-by-order velocities using `results.combine_orders('star')` and access the final (non-barycentric-corrected RVs as `results.star_time_rvs`.

In [ ]:
plt.errorbar(results2.dates, results2.star_rvs[r] + results2.bervs, 
             1./np.sqrt(results2.star_ivars_rvs[r]), 
             fmt='o', ms=5, elinewidth=1)
plt.xlabel('JD')
plt.ylabel(r'RV (m s$^{-1}$)')
plt.xlim([2456505, 2456570]);

### Now let's generalize this to multiple orders and get RVs for the full* spectrum:

##### * not actually the complete spectrum in this tutorial because that would take a long time to run

Here we'll overwrite the Data object with one that contains only a subset of spectral orders. The following commands could (and should) be used on the entire object containing all orders, but that can take a long time (up to an hour) so for the sake of the tutorial we'll use a smaller subsample.

In [ ]:
data = wobble.Data('../data/51peg_e2ds.hdf5', orders=np.arange(65,70))

In the following loop, we'll continually be overwriting the "model" variable. That's ok! All optimized results will be copied over to the "results" object automatically, and as long as the star component is given the same name in the model for every order, they'll be associated in the results object as we'd expect.

In [ ]:
results = wobble.Results(data=data)
for r in range(len(data.orders)):
        print('starting order {0} of {1}'.format(r+1, len(data.orders)))
        model = wobble.Model(data, results, r)
        model.add_star('star')
        model.add_telluric('tellurics', variable_bases=2)
        wobble.optimize_order(model)

Now that we have RVs, let's do some post-processing on them:

In [ ]:
results.combine_orders('star')

In [ ]:
results.apply_drifts('star') # instrumental drift corrections
results.apply_bervs('star') # barycentric corrections

Finally, we can look at the resulting RVs and save them to a text file:

In [ ]:
plt.errorbar(data.dates, results.star_time_rvs - np.mean(results.star_time_rvs), 
                     results.star_time_sigmas,
                    fmt='o', ms=5, elinewidth=1)
plt.xlabel('JD')
plt.ylabel(r'RV (m s$^{-1}$)');

In [ ]:
results.write_rvs('star', 'demo_rvs.txt')

The above command saved the RVs; we probably also want to save the spectral fits and other diagnostic information for future reference. We can do that with the following command, which preserves the entire Results object (including RVs and spectra):

In [ ]:
results.write('demo_results.hdf5')

### Further exercises:
- [optimize the regularization amplitudes](https://github.com/megbedell/wobble/blob/master/scripts/regularization.py) and rerun
- check [the API](https://wobble.readthedocs.io/en/latest/api.html) and figure out what keywords to use if you want to:
    - calculate uncertainties on the stellar template (warning: this will run slowly)
    - give your own custom starting RV guess to a component